In [2]:
#libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import normalize

In [3]:
#read the data 
recipes = pd.read_csv('../data/full_clean_data.csv')
#rename the index
recipes.index.names = ['recipes_id']
recipes.tail(2)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
recipes_id,,,,,,,,,,,,,,
19222,Jalapeno Hummus,https://www.allrecipes.com/recipe/46462/jalape...,appetizers-and-snacks,4.56,281,"['1 cup garbanzo beans ', ' ⅓ cup canned jalap...",10,8,8 servings,74.6,9.1,3.5,2.6,"['garbanzo bean', 'jalapeno reserved', 'tahini..."
19223,Easy Baked Zucchini Chips,https://www.allrecipes.com/recipe/242066/easy-...,appetizers-and-snacks,3.68,28,"['1 zucchini, thinly sliced ', ' 1 tablespoon...",30,2,2 servings,69.1,2.0,6.9,0.7,['flavored']


In [26]:
##create a new df for user input & concate it to the original dataframe
#user input: a = total time, b = calories, c = carbohydrate, d = fat, e = protein.
def user_recipes(a, b, c, d, e):
    user_df = pd.DataFrame({'recipes_id': 19224 ,'name': 'user', 'url': 'no', 'category': 'no', 'rating': 'no', 'rating_count': 'no', 'ingredients': 'no', 'total': a, 'servings': 'no', 'yield': 'no', 'calories': b, 'carbohydrates_g': c, 'fat_g': d,'protein_g': e,'ingredients_parsed': 'no'}, index=[19224])
    full_df = pd.concat([recipes, user_df], ignore_index = False, axis = 0) 
    #rename the dataframe index
    full_df.index.names = ['recipes_id']
    ##convert index to column
    full_df['recipes_id'] = full_df.index
    ##normalize the total, calories, carb, fat, protein features
    new_norm = full_df[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
    full_norm = pd.DataFrame(normalize(new_norm, axis=0))
    full_norm.columns = new_norm.columns
    full_norm.index = new_norm.index
    #use the model function here
    global allRecipes #Solution for the UnboundLocalError that didn't work???
    allRecipes = pd.DataFrame(full_norm.index)
    #execlude user recipe id from the rest
    allRecipes = allRecipes[allRecipes.recipes_id != 19224]
    allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: euclidean(full_norm.loc[19224], full_norm.loc[x]))
    # sort by distance then recipe id, the smaller value of recipe id will be picked.
    Top3NRecommendation = allRecipes.sort_values(["distance"]).head(3).sort_values(by=['distance', 'recipes_id'])
    #create a list of the recommended recipe output
    recipe_list = []
    for recipeid in Top3NRecommendation.recipes_id:
        recipe_list.append("{}  {}".format(recipeid, full_df.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, full_df.at[recipeid, 'name']))
        recipe_list.append("{}  {}".format(recipeid, full_df.at[recipeid, 'total']))
        recipe_list.append("{}  {}".format(recipeid, full_df.at[recipeid, 'calories']))
        recipe_list.append("{}  {}".format(recipeid, full_df.at[recipeid, 'carbohydrates_g']))
        recipe_list.append("{}  {}".format(recipeid, full_df.at[recipeid, 'fat_g']))
        recipe_list.append("{}  {}".format(recipeid, full_df.at[recipeid, 'protein_g']))
    return recipe_list
    

In [27]:
##the user wants a recipe that takes 45 minutes to cook, 300 calories, 40 gm of carbs, 9 gm of fat, 8 gm of protein)
user_recipes(45, 300, 40, 9, 8)

['9927  https://www.allrecipes.com/recipe/82264/mediterranean-quinoa/',
 '9927  Mediterranean Quinoa',
 '9927  45',
 '9927  276.4',
 '9927  40.8',
 '9927  9.8',
 '9927  8.1',
 '7899  https://www.allrecipes.com/recipe/232844/sarahs-feta-rice-pilaf/',
 "7899  Sarah's Feta Rice Pilaf",
 '7899  40',
 '7899  295.2',
 '7899  41.0',
 '7899  10.6',
 '7899  8.8',
 '2587  https://www.allrecipes.com/recipe/238162/vegan-black-bean-and-sweet-potato-salad/',
 '2587  Vegan Black Bean and Sweet Potato Salad',
 '2587  40',
 '2587  291.1',
 '2587  42.2',
 '2587  10.6',
 '2587  8.4']